In [1]:
import pandas_gbq
import pandas as pd
import json 
import datetime as dt
import requests
import numpy as np
from sklearn import preprocessing

#### Using Bigquery extract data order of user in 50 first days (from first order) in timerange (1/1/2019-20/2/2020)

In [2]:
df=pd.read_csv('./churn_han.csv')

In [5]:
df.head()

,id,comment_by_user,complete_time,discount,order_time,promo_code,rating_by_user,service_id,status,total_fee,total_pay,user_bonus_account,user_main_account,path_length,cancel_comment,partner_discount,partner_fee,first_completed,category,source
0,NPOH55,NaN,2019-05-08 10:53:31+00:00,0.0,2019-05-07 22:11:40+00:00,NaN,NaN,HAN-BIKE,COMPLETED,33000.0,33000.0,0.0,0.0,2,NaN,NaN,NaN,2019-04-24 10:48:21+00:00,Small business - Others,Direct Sale
1,Q131Z6,NaN,2019-05-08 18:21:51+00:00,0.0,2019-05-07 15:30:52+00:00,NaN,NaN,HAN-POOL,COMPLETED,34000.0,34000.0,0.0,0.0,2,NaN,NaN,NaN,2019-05-07 15:30:52+00:00,Small business - Accessories,Direct Sale
2,GO0FJG,NaN,2019-05-02 08:59:50+00:00,0.0,2019-05-01 20:55:58+00:00,NaN,5.0,HAN-BIKE,COMPLETED,26000.0,26000.0,0.0,0.0,2,NaN,NaN,NaN,2019-04-30 11:18:39+00:00,Small business - Souvenir,Organic
3,EK77AR,NaN,2019-04-26 06:41:50+00:00,0.0,2019-04-25 15:45:10+00:00,NaN,NaN,HAN-BIKE,COMPLETED,126000.0,126000.0,0.0,0.0,4,NaN,NaN,NaN,2019-03-15 09:03:07+00:00,Small business - Fashion,Direct Sale
4,FCX4DO,NaN,2019-05-10 11:09:05+00:00,0.0,2019-05-09 16:27:52+00:00,NaN,NaN,HAN-DG,COMPLETED,90000.0,90000.0,0.0,0.0,4,NaN,NaN,NaN,2019-05-09 09:28:25+00:00,Small business - Accessories,Direct Sale


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505066 entries, 0 to 505065
Data columns (total 22 columns):
id                    505066 non-null object
comment_by_user       9200 non-null object
complete_time         398673 non-null object
discount              505066 non-null float64
order_time            505066 non-null object
promo_code            142214 non-null object
rating_by_user        101470 non-null float64
service_id            505066 non-null object
status                505066 non-null object
total_fee             505066 non-null float64
total_pay             505066 non-null float64
user_bonus_account    505066 non-null float64
user_main_account     505066 non-null float64
user_id               505066 non-null int64
path_length           505066 non-null int64
cancel_comment        106533 non-null object
partner_discount      54518 non-null float64
partner_fee           54518 non-null float64
main_id               505066 non-null int64
first_completed       505066 non-

## clean data & extract feature

In [8]:
# convert string to datetime timezone GMT+7
df['order_time']=df['order_time'].astype('datetime64[ns]')+dt.timedelta(hours=7)
df['complete_time']=df['complete_time'].astype('datetime64[ns]')
df['first_completed']=df['first_completed'].astype('datetime64[ns]')+dt.timedelta(hours=7)
# xét cut_off time là first_completed +50 day.
df['cutoff_time']=df['first_completed']+dt.timedelta(days=50)
df['cutoff_date']=df['cutoff_time'].apply(lambda x:x.strftime('%Y-%m-%d')).astype('datetime64')
df['order_date']=df['order_time'].apply(lambda x:x.strftime('%Y-%m-%d')).astype('datetime64')
df['week_7']=df['cutoff_date'].astype('datetime64[ns]')-dt.timedelta(days=7)
df['week_6']=df['week_7'].astype('datetime64[ns]')-dt.timedelta(days=7)
df['week_5']=df['week_6'].astype('datetime64[ns]')-dt.timedelta(days=7)
df['week_4']=df['week_5'].astype('datetime64[ns]')-dt.timedelta(days=7)
df['week_3']=df['week_4'].astype('datetime64[ns]')-dt.timedelta(days=7)
df['week_2']=df['week_3'].astype('datetime64[ns]')-dt.timedelta(days=7)
df['week_1']=df['week_2'].astype('datetime64[ns]')-dt.timedelta(days=7)

In [11]:
# build feature

In [ ]:
df1=df[(df.order_time<df.cutoff_date)]
df2=df1[df1.status=='COMPLETED']
df2['stop']=df2.path_length -1
df2['total_fee']=np.where(df2.total_fee==0,df2.partner_fee,df2.total_fee)
df2['total_fee']=df2['total_fee'].astype('float64')
df2['discount_f']=np.where((df2.discount<0)|((df2.discount==0)&(df2.partner_discount>0)),df2.partner_discount,df2.discount)
# convert date to the beginning of the week
df2['order_week']=df2.order_date.apply(lambda x:x -dt.timedelta(days=x.weekday()))
df2['discount_f']=np.where((df2.discount<0)|((df2.discount==0)&(df2.partner_discount>0)),df2.partner_discount,df2.discount)
df2['main_id']=df2.main_id.astype('str')
# build features (order, active_day, stop, active_week, total_fee, total_discount, total_bonus_account)
df12=df2.groupby('main_id').agg({'id':'count','order_date':pd.Series.nunique,'stop':'sum','order_week':pd.Series.nunique,'total_fee':'sum','discount_f':'sum','user_bonus_account':'sum'}).reset_index().rename(columns={'id':'order','order_date':'active_day','order_week':'active_week','discount_f':'total_discount','user_bonus_account':'total_bonus_account'})
## order_discount
df13=df2[df2.discount_f>0].groupby('main_id').agg({'id':'count'}).reset_index().rename(columns={'id':'order_discount'})
## recency: last comeplete đến cutoff time
df3=df2.groupby(['main_id','cutoff_date']).agg({'complete_time':'max'}).reset_index().rename(columns={'complete_time':'last_complete'})
df3['recency']=(df3['cutoff_date']-df3['last_complete']).dt.days
## cancel order , cancel order by supplier
df14=df[df.status=='CANCELLED'].groupby('main_id').agg({'id':'count'}).reset_index().rename(columns={'id':'cancel_order'})
## rating
df15=pd.DataFrame(df[~df.rating_by_user.isna()].groupby(['main_id','rating_by_user']).agg({'id':'count'}).reset_index().pivot('main_id','rating_by_user','id').to_records())
## order theo each week (7 ngày)
dfw0=df2[(df2.order_time<df2.week_1)].groupby('main_id').agg({'id':'count'}).reset_index().rename(columns={'id':'week0_order'})
dfw1=df2[(df2.order_time>=df2.week_1)&(df2.order_time<df2.week_2)].groupby('main_id').agg({'id':'count'}).reset_index().rename(columns={'id':'week1_order'})
dfw2=df2[(df2.order_time>=df2.week_2)&(df2.order_time<df2.week_3)].groupby('main_id').agg({'id':'count'}).reset_index().rename(columns={'id':'week2_order'})
dfw3=df2[(df2.order_time>=df2.week_3)&(df2.order_time<df2.week_4)].groupby('main_id').agg({'id':'count'}).reset_index().rename(columns={'id':'week3_order'})
dfw4=df2[(df2.order_time>=df2.week_4)&(df2.order_time<df2.week_5)].groupby('main_id').agg({'id':'count'}).reset_index().rename(columns={'id':'week4_order'})
dfw5=df2[(df2.order_time>=df2.week_5)&(df2.order_time<df2.week_6)].groupby('main_id').agg({'id':'count'}).reset_index().rename(columns={'id':'week5_order'})
dfw6=df2[(df2.order_time>=df2.week_6)&(df2.order_time<df2.week_7)].groupby('main_id').agg({'id':'count'}).reset_index().rename(columns={'id':'week6_order'})
dfw7=df2[(df2.order_time>=df2.week_7)&(df2.order_time<df2.cutoff_date)].groupby('main_id').agg({'id':'count'}).reset_index().rename(columns={'id':'week7_order'})
df14['main_id']=df14.main_id.astype('str')
df15['main_id']=df15.main_id.astype('str')
df20=pd.merge(df12,df13,how='left',on='main_id').\
    merge(df14,how='left',on='main_id').\
    merge(df15,how='left',on='main_id').\
    merge(df16,how='left',on='main_id').\
    merge(df17,on='main_id')
## category
df20['category_1']=df20.category.str.split(' - ',expand = True)[0]
df20['category_1']=np.where(df20.category_1.isin(['Small business','SMEs',' vé khóa học']),'Small business',df20['category_1'])
df20['category_1']=np.where(~df20.category_1.isin(['Small business','Corporation']),'Individual',df20['category_1'])
b=['Food & Drink', 'Other', 'Fashion & Sport', 'Health & Beauty', 'Home & Lifestyle', 'Electronics', 'Mother & Baby', 'Transportation','Individual']
df20['category_2']=df20.category.str.split(' - ',expand = True)[1]
df20['category_2']=np.where(df20.category_1=='Individual','Individual',df20['category_2'])
df20['category_2']=df20.category_2.fillna('Individual')
df20['category_2']=np.where(df20.category_2.str.contains('food|Cake|Drink'),'Food & Drink',df20['category_2'])
df20['category_2']=np.where(df20.category_2.str.contains('Electronic'),'Electronics',df20['category_2'])
df20['category_2']=np.where(df20.category_2.str.contains('Fashion'),'Fashion & Sport',df20['category_2'])
df20['category_2']=np.where(df20.category_2.str.contains('Accessories|Souvenir|Interior decorations'),'Home & Lifestyle',df20['category_2'])
df20['category_2']=np.where(df20.category_2.str.contains('Cosmetic|Flower'),'Health & Beauty',df20['category_2'])
df20['category_2']=np.where(~df20.category_2.isin(b),'Other',df20['category_2'])
df21=df20.merge(dfw0,how='left',on='main_id').\
    merge(dfw1,how='left',on='main_id').\
    merge(dfw2,how='left',on='main_id').\
    merge(dfw3,how='left',on='main_id').\
    merge(dfw4,how='left',on='main_id').\
    merge(dfw5,how='left',on='main_id').\
    merge(dfw6,how='left',on='main_id').\
    merge(dfw7,how='left',on='main_id').\
    merge(df3[['main_id','recency']].drop_duplicates(),on='main_id')
df21=df21.drop(columns={'category'})
df22=df21.fillna(0)
# source
df22['source']=np.where(df22.source!='Direct Sale','Organic',df22.source)

In [ ]:
def toNumeric(dataframe, fromAtt, toAtt):
  if dataframe[fromAtt].dtype == type(object):
      le = preprocessing.LabelEncoder()# Dummies / One Hot Encoder  
      dataframe[toAtt] = le.fit_transform(dataframe[fromAtt].astype(str))   
  return dataframe

In [ ]:
## category feature
df22=toNumeric(df22, 'category_1', 'n_category_1')
df22=toNumeric(df22, 'source', 'n_source')
df22=toNumeric(df22, 'category_2', 'n_category_2')

In [ ]:
## label
a1=df[(df.order_time>=df.cutoff_date)&(df.status=='COMPLETED')].main_id.astype('str').drop_duplicates().tolist()
df22['label']=np.where(df22.main_id.isin(a1),1,0)

#### Total 28k user chỉ có 8639 user có lên đơn vào 7 ngày đầu tiên sau khi qua happiness. Chiếm 1/3 user

In [13]:
# print dataframe result (hide user_id field)
df22

,order,active_day,stop,active_week,total_fee,total_discount,total_bonus_account,order_discount,cancel_order,1.0,...,week3_order,week4_order,week5_order,week6_order,week7_order,recency,label,n_category_1,n_source,n_category_2
0,1,1,1,1,27000.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,49,0,2,1,1
1,10,8,10,5,327000.0,15000.0,0.0,3.0,6.0,0.0,...,2.0,3.0,2.0,1.0,0.0,13,0,2,0,2
2,1,1,1,1,54000.0,10000.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,49,0,2,0,6
3,8,6,8,4,370000.0,0.0,0.0,0.0,2.0,0.0,...,0.0,3.0,0.0,0.0,2.0,0,1,2,1,0
4,28,16,28,6,710000.0,0.0,0.0,0.0,4.0,0.0,...,2.0,7.0,1.0,1.0,0.0,13,1,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27579,4,4,4,4,128000.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1,0,1,1,5
27580,3,3,3,3,189000.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,1.0,1.0,0.0,7,1,2,1,7
27581,1,1,1,1,36000.0,6000.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,49,0,2,1,3
27582,2,2,2,2,45000.0,4000.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,34,0,1,1,5


In [15]:
df22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27584 entries, 0 to 27583
Data columns (total 31 columns):
main_id                27584 non-null int64
order                  27584 non-null int64
active_day             27584 non-null int64
stop                   27584 non-null int64
active_week            27584 non-null int64
total_fee              27584 non-null float64
total_discount         27584 non-null float64
total_bonus_account    27584 non-null float64
order_discount         27584 non-null float64
cancel_order           27584 non-null float64
1.0                    27584 non-null float64
2.0                    27584 non-null float64
3.0                    27584 non-null float64
4.0                    27584 non-null float64
5.0                    27584 non-null float64
source                 27584 non-null object
category_1             27584 non-null object
category_2             27584 non-null object
week0_order            27584 non-null float64
week1_order            27584 n

#### After clean data and extract feature, 27584 rows (27584 users) and 30 columns (30 features)